# Optimizing a single point (n-dimensional pitch aggregate)

This noteboook demonstrates how to find the optimal tuning for a single point. In this case, a "point" is a pitch aggregate in n-dimensional space, where the pitch aggregate contains n+1 pitches. For example, a triad would be 2-dimensional. The values of the dimensions are equal to the base-2 logarithms of the ratios of the frequencies from one of the pitches (which we might call the "root").

## Setup and Usage

The next few cells import the required tools and set some basic variables.

In [2]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import tensorflow as tf

devices = tf.config.experimental.get_visible_devices('GPU')
if len(devices) > 0:
    tf.config.experimental.set_memory_growth(devices[0], True)

In [2]:
import harmonic_distance as hd
import numpy as np

## Global Constants

These constants should be scaled appropriately. Comments are in-line.

In [18]:
# The "curve" of the parabola around each possible pitch. A higher 
# value will lead to fewer possible pitches.
C = 0.01

# The learning rate of the optimization algorithm. A higher value 
# will converge more quickly, if possible, but might never converge.
LEARNING_RATE = 1.0e-3

# The convergence threshold is the norm of the gradients of the loss
# function. This is used to test whether a proper "valley" has been found.
CONVERGENCE_THRESHOLD = 1.0e-3

## Creating the vector space

The _vector space_ can be thought of as the list of "all possible pitches."

In this implementation, we're using a `VectorSpace` subclass of [`tf.Module`](https://www.tensorflow.org/api_docs/python/tf/Module?version=stable) to cache the variables that do not change through each iteration.

### `hd.vectors.space_graph_altered_permutations`

The array `[5, 5, 3, 3, 2, 1]` sets the number of degrees along each dimension of the harmonic lattice (Tenney 19XX) that are available for tuning. The `bounds` value of `(0.0, 4.0)` restricts these harmonic possibilities to a 4-octave range in pitch space.

### `hd.tenney.hd_aggregate_graph`

Calculates the harmonic distance of every vector.

### `vectors_reasonable`

Restricts the possible pitches to values with a harmonic distances less than `9.0`. This is useful for the purposes of reducing memory consumption.

## Setting starting values

The starting values are in base-2 logarithmic form. These values listed below are a perfect fourth and a minor seventh above a given root.

In [19]:
class Minimizer(tf.Module):
    def __init__(self):
        starting_values = [5.0 / 12.0, 10.0 / 12.0]
        self.opt = tf.optimizers.Adagrad(learning_rate=LEARNING_RATE)
        self.vs = hd.vectors.VectorSpace(dimensions=1)
        self.log_pitches = tf.Variable(starting_values, dtype=tf.float64)
    
    @tf.function
    def minimize(self):
        while self.stopping_op():
            self.opt.minimize(lambda: self.loss(self.log_pitches), self.log_pitches)
            
    @tf.function
    def loss(self, var_list):
        return hd.optimize.parabolic_loss_function(self.vs.pds, self.vs.hds, var_list, curves=(C, C))
            
    @tf.function
    def stopping_op(self):
        with tf.GradientTape() as g:
            dz_dv = g.gradient(self.loss(self.log_pitches), self.log_pitches)
        norms = tf.nn.l2_loss(dz_dv)
        return norms >= CONVERGENCE_THRESHOLD

In [20]:
minimizer = Minimizer()

In [23]:
minimizer.log_pitches.assign([4.0 / 12.0, 10.0 / 12.0])
minimizer.minimize()
minimizer.log_pitches

<tf.Variable 'Variable:0' shape=(2,) dtype=float64, numpy=array([0.32192809, 0.84799194])>

In [22]:
np.log2([9.0 / 5.0])

array([0.84799691])